In [1]:
# Import
import pandas as pd
import os
import numpy as np
from sklearn.metrics import pairwise_distances

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from dash import Dash, dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash



from eeg_preictal_analysis import EEGPreictalAnalysis
eeg_class = EEGPreictalAnalysis()

feature_group = 'Multivariate' # 'Multivariate' OR 'Univariate_linear' OR 'Univariate_nonlinear'

In [2]:
## Prepare data to run feature reduction
patient_index = 112802
seizure_index = 6
pat_seiz_data_folder = 'patient' + str(patient_index) + '_seizure' + str(seizure_index)

# get current working directory
wd = os.getcwd()

# define dictionary for saving prepared features (save_flag=1 to save and save_flag=0 otherwise)
save_dict = {'save_flag': 0, 'save_folder': ''}

# get time data
df_datetime_vector, indexes_5min_win2remove = eeg_class.prepare_time_data4feat_reduction(patient_index, seizure_index, pat_seiz_data_folder, save_dict)
df_datetime_vector.head()

# load patient information to get seizure onset date
eeg_onset_seizure = eeg_class.get_seizure_onset(patient_index, seizure_index)

df_time_min, df_time_h = eeg_class.get_time_data(df_datetime_vector, 'win_start_date', eeg_onset_seizure)


get_time_vector = 'min'

if get_time_vector == 'min':
    time_vec_minutes = df_time_min['time_min']
    time_vector = time_vec_minutes*(-1)
    tickvals_lst = [-10, -50, -100, -150, -200, time_vector[0]]
    ticktext_lst = [-10, -50, -100, -150, -200, round(time_vector[0])]
else:
    time_vector = eeg_class.update_time_hour_night_transition(df_time_h)

# prepare feature data for feature reduction
df_seizure_data, original_feat_names, constant_feat_names, quasi_constant_feat_names = eeg_class.prepare_feature_data4feat_reduction(pat_seiz_data_folder, feature_group, indexes_5min_win2remove, save_dict)
df_seizure_data.head()


,Circular_correlation_theta_MS,WPLI_theta_A,Circular_correlation_alpha_A,Circular_correlation_gamma4_M,corr_max_xcorr_gamma1_MS,PSI_theta_M,PSI_beta_M,WPLI_delta_WGCC,dWPLI_theta_M,corr_max_xcorr_beta_MS,...,corr_max_xcorr_power_gamma1_MS,dWPLI_alpha_MS,ISPC_gamma1_M,WPLI_gamma2_CPL,Circular_correlation_gamma3_MCC,Circular_correlation_gamma2_GE,Circular_correlation_theta_WGCC,ISPC_gamma3_WGCC,ISPC_theta_M,ISPC_gamma2_WGCC
0,0.386524,0.875039,0.092866,-0.012882,-0.447170,0.002367,-0.022345,-0.586324,-0.278149,-0.690156,...,-0.540962,-0.695899,0.910444,-0.535316,1.028426,-0.038967,0.544772,-0.885291,0.463175,-0.196247
1,-1.248267,0.883137,0.534454,0.375969,-0.618571,0.002286,-0.024906,1.426013,1.463987,-0.440878,...,-0.403440,-0.361738,0.136503,-0.174264,1.185835,-0.162812,-1.157076,-0.885291,1.433422,-0.561990
2,-1.139668,0.858192,1.613167,1.286631,-0.360051,0.002220,-0.021949,-1.347149,1.371240,-0.818520,...,-0.470733,-0.539755,0.939651,0.286355,1.300484,-0.759442,-1.055721,-0.885291,1.683544,-0.911392
3,-0.353413,0.156531,0.628690,-0.650807,-0.534277,0.002248,-0.022954,-0.924154,-0.002048,-0.569640,...,-0.540962,-0.695899,0.863091,-1.174883,-0.320576,-0.040507,-0.600721,-0.232456,0.533717,-0.463163
4,-0.632802,-0.308169,-1.228111,0.351604,-0.532044,0.002354,-0.022189,-0.140931,0.624035,-0.815843,...,-0.540962,0.457747,0.102679,-0.677033,0.384539,0.275331,-0.721538,-0.410362,1.450525,0.320196


In [3]:
## Apply feature reduction, apply clustering methods and evaluate clustering solutions

clust_methods = ['KMEANS_K2', 'KMEANS_K3', 'KMEANS_K4', 'AGGLO_HIER_K2', 'AGGLO_HIER_K3', 'AGGLO_HIER_K4', 'HDBSCAN0', 'GMM_K2', 'GMM_K3', 'GMM_K4']

clust_eval = ['clust_method', 'n_clusters', 'noisy_clusters', 'n_samples_smaller_cluster', 'DI', 'SI', 'OD', 'CS', 'C', 'DBI']

print(clust_eval)

['clust_method', 'n_clusters', 'noisy_clusters', 'n_samples_smaller_cluster', 'DI', 'SI', 'OD', 'CS', 'C', 'DBI']


In [4]:
## Apply Principal Component Analysis (PCA):
df_pca, pca = eeg_class.pca_feature_reduction(df_seizure_data)
df_pca.head()
distances_matrix = pairwise_distances(df_pca)


In [5]:
## Apply user input clustering method and plot results

# decided to use dash as no other widget would allow to show input and dropdown menus

# creating the layout
# initialize the app
app = Dash(__name__)

dropdown_option_labels = ['HDBSCAN', 'K-means', 'Agglomerative Hierarchical', 'Gaussian Mixture Models', 'DBSCAN']
dropdown_option_values = ['HDBSCAN0', 'KMEANS', 'AGGLO_HIER', 'GMM', 'DBSCAN']

app.layout = html.Div(id = 'parent', children = [

    # setting the graph component
    dcc.Graph(id = 'scatter-plot'),

    html.Div([
        dbc.Row([
            dbc.Col(html.Div(
                dbc.Label(['Choose clustering algorithm:'], style={'font-weight': 'bold'})
            ), width=6, lg=3),

            dbc.Col(html.Div(
                dcc.Dropdown(id = 'dropdown',
                             options = [{'label':i, 'value':j} for i, j in zip(dropdown_option_labels, dropdown_option_values)],
                             value = 'KMEANS',
                             style = {'width':'50%', 'verticalAlign': 'middle'}, # , 'height':'auto',  'font-size' : '12px'
                             placeholder = 'Select a clustering algorithm...')
            ), width={"size": 3, "order": "last", "offset": 1}, lg=3)
        ]),

        dbc.Row([
            dbc.Col(html.Div(
                dbc.Label(['Input number of clusters: '], style={'font-weight': 'bold'})
            ), width=6, lg=3),

            dbc.Col(html.Div(
                dcc.Input(id='ncluster-input', value='2', type='text')
            ), width=6, lg=3)
        ]),
    ])

])

## 2 input components corresponding to input and dropdown respectively
## 1 output component corresponding to the scatter plot
@app.callback(Output(component_id='scatter-plot', component_property='figure'),
              [Input(component_id='ncluster-input', component_property='value'),
               Input(component_id='dropdown', component_property= 'value')])

def graph_update(n_cluster_value, clust_method_value):
    # filtering based on the input and dropdown selection
    if clust_method_value == 'HDBSCAN0':
        cluster_method = 'HDBSCAN'
        clustering_method = clust_method_value
        what2display = 'block'
    elif clust_method_value == 'DBSCAN':
        what2display = 'none'
        cluster_method = clust_method_value
        clustering_method = clust_method_value + '_D' + n_cluster_value
    else:
        what2display = 'none'
        clustering_method = clust_method_value + '_K' + n_cluster_value
        if clust_method_value == 'AGGLO_HIER':
            cluster_method = 'Agglomerative Hierarchical'
        elif clust_method_value == 'KMEANS':
            cluster_method = 'K-means'
        elif clust_method_value == 'GMM':
            cluster_method = 'Gaussian Mixture Models'

    print(cluster_method)
    print(clustering_method)
    # compute the default clustering solution with K-means, k = 2
    clustering_solution = eeg_class.perform_clustering(df_pca, clustering_method)
    eval_list = eeg_class.cluster_evaluation_indexes(df_pca, clustering_solution, distances_matrix)
    explained_variance = sum(pca.explained_variance_ratio_)

    pca_title = 'PCA feature reduction<br>(explained variance = ' + str(round(explained_variance*100, 1)) + '%)'
    cluster_title = cluster_method + ', k=' + n_cluster_value + ' clustering solution<br> '
    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'scene'}, {'type': 'scene'}]],
                        subplot_titles=(pca_title, cluster_title),
                        shared_yaxes=False, shared_xaxes=False, horizontal_spacing=0.05)

    fig.add_trace(go.Scatter3d(x=df_pca['PC1'], y=df_pca['PC2'], z=df_pca['PC3'], mode='markers',
                               marker=dict(size=3, color=time_vector, colorscale='Viridis', showscale=True,
                                           colorbar=dict(len=1, x=-0.15, y=0.45, titleside='right', thickness=20,
                                                         title='<b>Time before seizure (' + get_time_vector +')<b>', outlinewidth=1, titlefont=dict(size=12), tickmode='array',
                                                         tickvals=tickvals_lst, ticktext=ticktext_lst,
                                                         ticklabelposition='outside'))))

    # plot the clustering solution
    fig.add_trace(go.Scatter3d(x=df_pca['PC1'], y=df_pca['PC2'], z=df_pca['PC3'], mode='markers',
                               marker=dict(size=3, color=clustering_solution, colorscale='Viridis')), row=1, col=2)

    fig.update_layout(width=1000, margin=dict(l=100, r=150, b=70, t=70),
                      title=dict(text='<b>Patient ' + str(patient_index) + ', seizure ' + str(seizure_index),
                                 y=0.98, x=0.47, xanchor='center', yanchor='top'),
                      template='plotly_white', showlegend=False,
                      scene = dict(xaxis_title='C1', yaxis_title='C2',zaxis_title='C3'),
                      scene2 = dict(xaxis_title='C1', yaxis_title='C2',zaxis_title='C3'))

    return fig

if __name__ == '__main__':
    app.run_server(debug=False,
                   # mode='inline'
                   )

stop


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [14/Mar/2023 19:17:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 19:17:56] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_8_1m1675174511.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 19:17:56] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_8_1m1675174511.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 19:17:56] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_4_0m1677548663.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 19:17:56] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_8_1m1675174511.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 19:17:56] "GET /_dash-component-suites/dash/deps/react@16.v2_8_1m1675174511.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 19:17:56] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_8_1m1675174511.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 19:17

K-means
KMEANS_K2


127.0.0.1 - - [14/Mar/2023 19:17:56] "POST /_dash-update-component HTTP/1.1" 200 -


NameError: name 'stop' is not defined

In [ ]:
# t-SNE
list_perplexity = np.arange(10, 110, 10)
tc1 = []
tc2 = []
tc3 = []
dic_tsne_clust_results = []
for perplexity in list_perplexity:

    df_tsne, array_tsne = eeg_class.tsne_feature_reduction(df_seizure_data, perplexity)

    tc1.append(array_tsne[:, 0])
    tc2.append(array_tsne[:, 1])
    tc3.append(array_tsne[:, 2])

    # get pairwise distances between samples
    distances_matrix = pairwise_distances(df_tsne)

    # start_time = datetime.now()
    for cm in clust_methods:
        # print({p}, {s}, {cm}, 'tsne', {perplexity})
        clustering_solution = eeg_class.perform_clustering(df_tsne, cm)

        eval_list = eeg_class.cluster_evaluation_indexes(df_tsne, clustering_solution, distances_matrix)
        dic_tsne_clust_results.append([perplexity, cm.lower()] + eval_list)

lst_string_perplexity = list(map(str, list_perplexity.tolist()))
df_tc1 = pd.DataFrame(tc1).transpose()
df_tc1.columns = lst_string_perplexity
df_tc2 = pd.DataFrame(tc2).transpose()
df_tc2.columns = lst_string_perplexity
df_tc3 = pd.DataFrame(tc3).transpose()
df_tc3.columns = lst_string_perplexity
df_tsne_clust_results = pd.DataFrame(dic_tsne_clust_results, columns=['perplexity'] + clust_eval)


In [ ]:
# UMAP
n_neighbours_vec = np.arange(10, 110, 10)
min_dist_vec = np.round(np.arange(0.1, 1.0, 0.1), 2)
uc1 = []
uc2 = []
uc3 = []
dic_umap_clust_results = []
lst_header = []
for n_neighbours in n_neighbours_vec:
    for min_dist in min_dist_vec:

        lst_header.append(str(n_neighbours) + '_' + str(min_dist))

        df_umap, array_umap = eeg_class.umap_feature_reduction(df_seizure_data, n_neighbours, min_dist)

        uc1.append(array_umap[:, 0])
        uc2.append(array_umap[:, 1])
        uc3.append(array_umap[:, 2])

        # get pairwise distances between samples
        distances_matrix = pairwise_distances(df_umap)

        # start_time = datetime.now()
        for cm in clust_methods:
            # print({p}, {s}, {cm}, 'umap', {n_neighbours}, {min_dist})
            clustering_solution = eeg_class.perform_clustering(df_umap, cm)

            eval_list = eeg_class.cluster_evaluation_indexes(df_umap, clustering_solution, distances_matrix)

            dic_umap_clust_results.append([n_neighbours, min_dist, cm.lower()] + eval_list)
            # print('end clustering')
        # print('end all clustering methods in umap')

df_uc1 = pd.DataFrame(uc1).transpose()
df_uc1.columns = lst_header
df_uc2 = pd.DataFrame(uc2).transpose()
df_uc2.columns = lst_header
df_uc3 = pd.DataFrame(uc3).transpose()
df_uc3.columns = lst_header

df_umap_clust_results = pd.DataFrame(dic_umap_clust_results, columns=['n_neighbors', 'min_dist'] + clust_eval)